In [1]:
import pandas as pd
import json
import os
import subprocess
import bibtexparser
from tqdm import tqdm
import re
from flair.data import Sentence
from flair.nn import Classifier
from difflib import SequenceMatcher


/Users/quenzer/Desktop/Data Science/3. Semester 2023 WS/Data Analysis Project/A-blessing-or-a-curse-Analysis-of-scholarly-citations/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/quenzer/Desktop/Data Science/3. Semester 2023 WS/Data Analysis Project/A-blessing-or-a-curse-Analysis-of-scholarly-citations/.venv/lib/python3.12/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/quenzer/Desktop/Data Science/3. Semester 2023 WS/Data Analysis Project/A-blessing-or-a-curse-Analysis-of-scholarly-citations/.venv/lib/python3.12/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node

In [2]:
#TODO: write a function that normalizes the names of the authors from our annotations
# to fit the format generated by anystyle.io check below for examples and look for the pattern

In [3]:
def file_finder(file_str: str) -> str:
    """
    This function takes a file name and returns the path to the file in the all_data_articles.
    """
    title_doi = "../data/titles_doi.csv"
    folder_path = "../all_data_articles"
    
    #extract the doi from the file name
    doi = file_str.split("_")[-1].split(".")[0]

    # find the row in the csv file where the doi column ends with the doi
    df = pd.read_csv(title_doi)
    doi_row = df[df["DOI"].str.endswith(doi)]

    # extract the title from the row
    title_json = doi_row["Title"].values[0].replace(" ", "_") + ".json"

    for filename in os.listdir(folder_path):
        if filename.endswith(".json") and filename.startswith(title_json[:int(len(title_json)/3)]):
            return filename


In [4]:
def load_annotations(file_str: str) -> pd.DataFrame:
    """
    This function takes a file name and returns the annotations from the file.
    And also replaces missing values with None.
    """
    folder_path = "../data/annotated"

    file_path = os.path.join(folder_path, file_str)
    df = pd.read_excel(file_path)

    # replace missing values with None
    df = df.where(pd.notnull(df), None)

    # replace values marked with nan with None
    df = df.replace("nan", None)
    
    return df

In [5]:
def format_author_name(name):
    # TODO: probably need to handle more cases
    if name is None:
        return None
    if ' and ' in name:
        # Handle multiple authors
        authors = name.split(' and ')
        formatted_authors = [format_author_name(author) for author in authors]
        return ' and '.join(formatted_authors)
    else:
        parts = name.split()
        # Handle case where there is a middle initial
        if len(parts) == 3:
            return f"{parts[1]}, {parts[0]} {parts[2]}"
        # Handle case where there is no middle initial
        elif len(parts) == 2:
            return f"{parts[1]}, {parts[0]}"
        else:
            return name

In [6]:
def df_to_triplets(df: pd.DataFrame, format_author = True) -> set:
    """
    This function takes a dataframe and returns a set of triplets.
    """
    triplets = set()
    for i in range(len(df)):
        if format_author:
            triplet = (df.iloc[i]["Footnote"], format_author_name(df.iloc[i]["Authors"]), df.iloc[i]["Title"])
        else:
            triplet = (df.iloc[i]["Footnote"], df.iloc[i]["Authors"], df.iloc[i]["Title"])
        triplets.add(triplet)
    return triplets

In [7]:
def dict_to_triplets(extraction: dict) -> set:
    """
    Converts a dictionary of footnotes to a set of triplets
    """
    triplets = set()

    for number, references in extraction.items():
        for reference in references:
            author = reference[0]
            title = reference[1]

            if author == "":
                author = None

            if title == "":
                title = None
            
            triplets.add((int(number), author, title))

    return triplets

In [8]:
split_pattern = re.compile('; see |; | . See also | .See also |. See |, see')

In [9]:
def information_extraction(file_path: str) -> set:
    """
    This function takes a file path and returns a set of triplets.
    """
    path = "../all_data_articles"
    file_path = os.path.join(path, file_path)
    article = json.load(open(file_path, "r"))
    extraction = {}

    prev_footnote = None

    for number, footnote in tqdm(article["footnotes"].items()):

        # If the footnote is ibid, use the previous footnote
        if footnote.startswith("Ibid"):
            # do not fully replace ibid with previous footnote but rather prepend it since there might be other references after ibid
            footnote = prev_footnote + "; " + footnote.lstrip("Ibid. ")

        # Store the footnote for the next iteration
        prev_footnote = footnote
        
        references = split_pattern.split(footnote)

        author_title_list = []

        for reference in references:

            command = ['ruby', 'anystyle.rb', str(reference).strip()]
            bibtex = subprocess.run(command, stdout=subprocess.PIPE, text=True).stdout
            parsed_bibtex = bibtexparser.loads(bibtex).entries
            
            if parsed_bibtex:
                parsed_bibtex = parsed_bibtex[0]
            else:
                #print(f"No valid BibTeX entry found in: {bibtex}, set to empty dict")
                parsed_bibtex = {}
                
            if "note" in parsed_bibtex:
                continue
    
            # Extract title and author
            title = parsed_bibtex.get('title', parsed_bibtex.get('booktitle', None))
            author = parsed_bibtex.get('author', parsed_bibtex.get("editor", None))
    
            if author is not None or title is not None:
                # Append author and title pair to the list
                author_title_list.append([author, title])
        
        # Store the list in the extraction dictionary with the footnote number as the key
        extraction[number] = author_title_list

    return dict_to_triplets(extraction)

In [10]:
def calculate_scores(triplets, extractions):
    TP = len(triplets & extractions)  # Intersection of triplets and extractions
    FP = len(extractions - triplets)  # Elements in extractions but not in triplets
    FN = len(triplets - extractions)  # Elements in triplets but not in extractions

    recall = TP / (TP + FN) if TP + FN != 0 else 0
    precision = TP / (TP + FP) if TP + FP != 0 else 0
    f_score = 2 * (precision * recall) / (precision + recall) if precision + recall != 0 else 0

    return recall, precision, f_score

In [11]:
from fuzzywuzzy import fuzz

/Users/quenzer/Desktop/Data Science/3. Semester 2023 WS/Data Analysis Project/A-blessing-or-a-curse-Analysis-of-scholarly-citations/.venv/lib/python3.12/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [12]:
def calculate_similarity(str1, str2):
    #return SequenceMatcher(None, str1, str2).ratio()
    
    # https://pypi.org/project/fuzzywuzzy/
    # TODO: could also use token_sort_ratio since token_set_ratio might be too generous
    return fuzz.token_set_ratio(str1, str2) / 100

def evaluate_extraction(set1, set2, threshold=0.95):
    true_positives = 0
    false_positives = 0
    false_negatives = 0

    for triplet1 in set1:
        footnote1, author1, title1 = triplet1
        author1 = author1 if author1 is not None else ""
        title1 = title1 if title1 is not None else ""
        concat_str1 = str(author1) + " " + str(title1)
        found_match = False

        for triplet2 in set2:
            footnote2, author2, title2 = triplet2
            author2 = author2 if author2 is not None else ""
            title2 = title2 if title2 is not None else ""
            concat_str2 = str(author2) + " " + str(title2)

            # Check for footnote number and similarity
            if footnote1 == footnote2 and calculate_similarity(concat_str1, concat_str2) >= threshold:
                found_match = True
                break

        if found_match:
            true_positives += 1
        else:
            false_negatives += 1

    false_positives = len(set2) - true_positives

    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return precision, recall, f_score

In [13]:
path_annotations = "../data/annotated"

for filename in os.listdir(path_annotations):
    print(filename)
    title_json = file_finder(filename)
    df = load_annotations(filename)
    triplets = df_to_triplets(df, format_author=True)
    extraction = information_extraction(title_json)
    recall, precision, f_score = evaluate_extraction(triplets, extraction, threshold=0.9)
    print("\n")
    print(f"Recall: {recall}")
    print(f"Precision: {precision}")
    print(f"F-Score: {f_score}")
    print("-"* 50)
    print("\n")
    #break

Labels - https___doi.org_10.1093_ehr_cew052.xlsx


100%|██████████| 148/148 [01:20<00:00,  1.85it/s]




Recall: 0.8288770053475936
Precision: 0.7989690721649485
F-Score: 0.8136482939632547
--------------------------------------------------


Labels - https___doi.org_10.1093_ehr_cead107.xlsx


100%|██████████| 124/124 [01:32<00:00,  1.34it/s]




Recall: 0.6958762886597938
Precision: 0.6887755102040817
F-Score: 0.6923076923076924
--------------------------------------------------


Labels - https___doi.org_10.1093_ehr_cead080.xlsx


...



Recall: 0.8910891089108911
Precision: 0.8294930875576036
F-Score: 0.8591885441527447
--------------------------------------------------


Labels - https___doi.org_10.1093_ehr_cead103.xlsx


 21%|██        | 24/115 [00:19<01:09,  1.30it/s]Entry type thesis not standard. Not considered.
Entry type thesis not standard. Not considered.
100%|██████████| 115/115 [01:25<00:00,  1.34it/s]




Recall: 0.766497461928934
Precision: 0.7512437810945274
F-Score: 0.7587939698492462
--------------------------------------------------


Labels - https___doi.org_10.1093_ehr_ceab280.xlsx


100%|██████████| 100/100 [00:53<00:00,  1.86it/s]



Recall: 0.6890756302521008
Precision: 0.34893617021276596
F-Score: 0.46327683615819204
--------------------------------------------------




In [14]:
# TODO: implement a simple approach with a just regrex to extract the author and title, and a simple split with ";"

def extract_citations(file_path: str) -> set:
    path = "../all_data_articles"
    file_path = os.path.join(path, file_path)
    article = json.load(open(file_path, "r"))
    
    citations = set()
    prev_footnote = None

    for footnote_number, footnote_text in tqdm(article["footnotes"].items()):
        # If the footnote is ibid, use the previous footnote
        if footnote_text.startswith("Ibid"):
            footnote_text = prev_footnote
        
        prev_footnote = footnote_text

        # Split the footnote into individual citations
        individual_citations = split_pattern.split(footnote_text)
        
        for citation_text in individual_citations:
            # Regular expression to extract authors and titles
            # TODO: try a better pattern
            pattern = re.compile(r'^(.+?),\s+(.+?)[,|(]')

            match = pattern.match(citation_text)
            
            if match:
                author = match.group(1)
                title = match.group(2)
                citations.add((int(footnote_number), author, title))


    return citations

 

In [15]:
path_annotations = "../data/annotated"

for filename in os.listdir(path_annotations):
    print(filename)
    title_json = file_finder(filename)
    df = load_annotations(filename)
    triplets = df_to_triplets(df, format_author=False)
    extraction = extract_citations(title_json)
    recall, precision, f_score = evaluate_extraction(triplets, extraction, threshold=0.9)
    print("\n")
    print(f"Recall: {recall}")
    print(f"Precision: {precision}")
    print(f"F-Score: {f_score}")
    print("-"* 50)
    print("\n")

Labels - https___doi.org_10.1093_ehr_cew052.xlsx


  0%|          | 0/148 [00:00<?, ?it/s]

100%|██████████| 148/148 [00:00<00:00, 145957.44it/s]




Recall: 0.7613636363636364
Precision: 0.6907216494845361
F-Score: 0.7243243243243243
--------------------------------------------------


Labels - https___doi.org_10.1093_ehr_cead107.xlsx


100%|██████████| 124/124 [00:00<00:00, 125022.52it/s]


...



Recall: 0.9947643979057592
Precision: 0.8755760368663594
F-Score: 0.9313725490196079
--------------------------------------------------


Labels - https___doi.org_10.1093_ehr_cead103.xlsx


100%|██████████| 115/115 [00:00<00:00, 133724.69it/s]




Recall: 0.8212290502793296
Precision: 0.7313432835820896
F-Score: 0.7736842105263159
--------------------------------------------------


Labels - https___doi.org_10.1093_ehr_ceab280.xlsx


100%|██████████| 100/100 [00:00<00:00, 183077.43it/s]



Recall: 0.8145161290322581
Precision: 0.4297872340425532
F-Score: 0.5626740947075209
--------------------------------------------------




In [16]:
path_annotations = "../data/annotated"

for filename in os.listdir(path_annotations):
    print(filename)
    title_json = file_finder(filename)
    df = load_annotations(filename)
    triplets = df_to_triplets(df, format_author=False)
    extraction = extract_citations(title_json)
    recall, precision, f_score = evaluate_extraction(triplets, extraction, threshold=0.9)
    print("\n")
    print(f"Recall: {recall}")
    print(f"Precision: {precision}")
    print(f"F-Score: {f_score}")
    print("-"* 50)
    print("\n")
    #break

Labels - https___doi.org_10.1093_ehr_cew052.xlsx


100%|██████████| 148/148 [00:00<00:00, 198642.24it/s]




Recall: 0.7613636363636364
Precision: 0.6907216494845361
F-Score: 0.7243243243243243
--------------------------------------------------


Labels - https___doi.org_10.1093_ehr_cead107.xlsx


100%|██████████| 124/124 [00:00<00:00, 124992.48it/s]




Recall: 0.8430232558139535
Precision: 0.7397959183673469
F-Score: 0.7880434782608696
--------------------------------------------------


Labels - https___doi.org_10.1093_ehr_cead080.xlsx


...



Recall: 0.9947643979057592
Precision: 0.8755760368663594
F-Score: 0.9313725490196079
--------------------------------------------------


Labels - https___doi.org_10.1093_ehr_cead103.xlsx


100%|██████████| 115/115 [00:00<00:00, 145944.01it/s]




Recall: 0.8212290502793296
Precision: 0.7313432835820896
F-Score: 0.7736842105263159
--------------------------------------------------


Labels - https___doi.org_10.1093_ehr_ceab280.xlsx


100%|██████████| 100/100 [00:00<00:00, 168378.32it/s]



Recall: 0.8145161290322581
Precision: 0.4297872340425532
F-Score: 0.5626740947075209
--------------------------------------------------




In [17]:
# TODO: write a function that extracts the author and title from the footnote using a tagger (i.e. flair)

def tagger_information_extraction(file_path: str, tagger) -> set:
    path = "../all_data_articles"
    file_path = os.path.join(path, file_path)
    article = json.load(open(file_path, "r"))
    
    citations = set()
    prev_footnote = None

    for footnote_number, footnote_text in tqdm(article["footnotes"].items()):
        # If the footnote is ibid, use the previous footnote
        if footnote_text.startswith("Ibid"):
            footnote_text = prev_footnote
        
        prev_footnote = footnote_text

        # Split the footnote into individual citations
        individual_citations = split_pattern.split(footnote_text)

        for citation_text in individual_citations:
            
            author = None

            sentence = Sentence(citation_text)
            tagger.predict(sentence)
            for span in sentence.get_spans('ner'):
                if span.tag == "PERSON" or span.tag == "ORG":
                    if author is None:
                        author = span.text
                    else:
                        author += ("and " + span.text)
                if span.tag == "WORK_OF_ART":
                    citations.add((int(footnote_number), author, span.text))
                    author = None
            
                
    return citations


In [18]:
path_annotations = "../data/annotated"
# load the NER tagger
tagger = Classifier.load('ner-ontonotes-large')

for filename in os.listdir(path_annotations):
    print(filename)
    title_json = file_finder(filename)
    df = load_annotations(filename)
    triplets = df_to_triplets(df, format_author=False)
    extraction = tagger_information_extraction(title_json, tagger=tagger)
    recall, precision, f_score = evaluate_extraction(triplets, extraction, threshold=0.90)
    print("\n")
    print(f"Recall: {recall}")
    print(f"Precision: {precision}")
    print(f"F-Score: {f_score}")
    print("-"* 50)
    print("\n")

2024-01-03 14:09:46,373 SequenceTagger predicts: Dictionary with 76 tags: <unk>, O, B-CARDINAL, E-CARDINAL, S-PERSON, S-CARDINAL, S-PRODUCT, B-PRODUCT, I-PRODUCT, E-PRODUCT, B-WORK_OF_ART, I-WORK_OF_ART, E-WORK_OF_ART, B-PERSON, E-PERSON, S-GPE, B-DATE, I-DATE, E-DATE, S-ORDINAL, S-LANGUAGE, I-PERSON, S-EVENT, S-DATE, B-QUANTITY, E-QUANTITY, S-TIME, B-TIME, I-TIME, E-TIME, B-GPE, E-GPE, S-ORG, I-GPE, S-NORP, B-FAC, I-FAC, E-FAC, B-NORP, E-NORP, S-PERCENT, B-ORG, E-ORG, B-LANGUAGE, E-LANGUAGE, I-CARDINAL, I-ORG, S-WORK_OF_ART, I-QUANTITY, B-MONEY
Labels - https___doi.org_10.1093_ehr_cew052.xlsx


100%|██████████| 148/148 [00:20<00:00,  7.06it/s]




Recall: 0.7962085308056872
Precision: 0.865979381443299
F-Score: 0.8296296296296297
--------------------------------------------------


Labels - https___doi.org_10.1093_ehr_cead107.xlsx


100%|██████████| 124/124 [00:25<00:00,  4.81it/s]




Recall: 0.7055837563451777
Precision: 0.7091836734693877
F-Score: 0.707379134860051
--------------------------------------------------


Labels - https___doi.org_10.1093_ehr_cead080.xlsx


...



Recall: 0.7675675675675676
Precision: 0.6543778801843319
F-Score: 0.7064676616915423
--------------------------------------------------


Labels - https___doi.org_10.1093_ehr_cead103.xlsx


100%|██████████| 115/115 [00:23<00:00,  4.79it/s]




Recall: 0.695906432748538
Precision: 0.5920398009950248
F-Score: 0.6397849462365591
--------------------------------------------------


Labels - https___doi.org_10.1093_ehr_ceab280.xlsx


100%|██████████| 100/100 [00:17<00:00,  5.84it/s]



Recall: 0.6783216783216783
Precision: 0.4127659574468085
F-Score: 0.5132275132275133
--------------------------------------------------


